# PydanticAI fundamentals

In [35]:
from pydantic_ai import Agent

agent = Agent(model="google-gla:gemini-2.5-flash")

result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result    

AgentRunResult(output='**Name:** Linnea Karlsson\n**Role:** Cybersecurity Analyst\n**Company:** Ericsson (Stockholm)')

In [36]:
print(result.output)

**Name:** Linnea Karlsson
**Role:** Cybersecurity Analyst
**Company:** Ericsson (Stockholm)


# use pydantic to structure output

In [37]:
from pydantic import BaseModel, Field


class Employee(BaseModel):
    name: str = Field(description="first name and last name of the person")
    age: int = Field(gt=17, lt=67, description="age of a person")
    salary: int = Field(
        gt=30_000,
        lt=70_000,
        description="salary of the employee, the more senior the role, the higher salary",
    )
    role: str = Field(
        description="always include seniority level in the role (junior, senior, expert)"
    )
    description: str = Field(
        description="short description (max 2 sentences) of the role, include keywords"
    )


result = await agent.run(
    "Give me an IT employee working as data engineer in Sweden", output_type=Employee
)

result

AgentRunResult(output=Employee(name='Johan Andersson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes.'))

In [38]:
employee = result.output
employee

Employee(name='Johan Andersson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes.')

In [39]:
employee.description, employee.name

('Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes.',
 'Johan Andersson')

In [40]:
isinstance(employee, BaseModel)

True

In [41]:
employee.model_dump()

{'name': 'Johan Andersson',
 'age': 35,
 'salary': 65000,
 'role': 'Senior Data Engineer',
 'description': 'Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes.'}

In [42]:
employee.model_dump_json()

'{"name":"Johan Andersson","age":35,"salary":65000,"role":"Senior Data Engineer","description":"Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes."}'

In [43]:
print(employee.model_dump_json(indent=3))

{
   "name": "Johan Andersson",
   "age": 35,
   "salary": 65000,
   "role": "Senior Data Engineer",
   "description": "Designs, builds, and maintains scalable data pipelines. Responsible for data integration, data warehousing, and ensuring data quality for analytical purposes."
}


### simulate 10 employees

In [44]:
result = await agent.run(
    "Give me 10 employees in AI, data science, data analyst filed", 
    output_type=list[Employee]
)

result

AgentRunResult(output=[Employee(name='Alice Smith', age=23, salary=65000, role='Junior AI Engineer', description='Works on developing and deploying machine learning models under supervision. Focuses on data preprocessing and model experimentation.'), Employee(name='Bob Johnson', age=25, salary=68000, role='Junior Data Scientist', description='Assists in data analysis projects, builds basic predictive models, and interprets statistical results. Focuses on learning advanced analytics techniques.'), Employee(name='Carol White', age=22, salary=50000, role='Entry-level Data Analyst', description='Responsible for collecting and cleaning data. Creates simple reports and helps with data visualization.'), Employee(name='David Green', age=26, salary=69000, role='Junior AI Researcher', description='Conducts preliminary research in AI topics, assists with literature reviews, and contributes to small-scale AI projects.'), Employee(name='Eve Brown', age=24, salary=55000, role='Junior Data Analyst', 

In [45]:
for employee in result.output:
    print(f"{employee.name} is {employee.age} years, earns {employee.salary} and have role {employee.role}")

Alice Smith is 23 years, earns 65000 and have role Junior AI Engineer
Bob Johnson is 25 years, earns 68000 and have role Junior Data Scientist
Carol White is 22 years, earns 50000 and have role Entry-level Data Analyst
David Green is 26 years, earns 69000 and have role Junior AI Researcher
Eve Brown is 24 years, earns 55000 and have role Junior Data Analyst
Frank Black is 27 years, earns 67000 and have role Junior Data Scientist
Grace Lee is 21 years, earns 40000 and have role AI Engineering Intern
Harry King is 28 years, earns 60000 and have role Entry-level Data Science Analyst
Ivy Chen is 23 years, earns 66000 and have role Junior AI Engineer
Jack Wong is 25 years, earns 58000 and have role Junior Data Analyst


## More complex model

In [46]:
from typing import Literal

class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing"] = Field(description="name of department")
    description: str = Field(description="short description of the department")
    employees: list[Employee]
    
class Company(BaseModel):
    name: str = Field(description="name of the company")
    departments: list[Department]
    description: str = Field(description="describe company in two sentences")
    
result = await agent.run(
    ("Give 2-3 employees in each department",
    "Company shpuld be called CoolCompany and sell ice creams"
    ),
    output_type=Company
)



In [47]:
company = result.output
company.__dict__.keys()

dict_keys(['name', 'departments', 'description'])

In [48]:
company.name

'CoolCompany'

In [49]:
[department.name for department in company.departments]

['IT', 'HR', 'Marketing']

In [50]:
departments = [department for department in company.departments]
departments[0].employees

[Employee(name='Alice Wonderland', age=35, salary=65000, role='Senior Software Engineer', description="Leads software development projects and maintains company's digital platforms. Specializes in scalable backend systems."),
 Employee(name='Bob Thebuilder', age=25, salary=55000, role='Junior IT Support Specialist', description='Provides technical assistance and resolves IT-related issues for employees. Handles hardware and software troubleshooting.')]

In [51]:
company.model_dump()

{'name': 'CoolCompany',
 'departments': [{'name': 'IT',
   'description': 'Manages all technological infrastructure and software development for the company.',
   'employees': [{'name': 'Alice Wonderland',
     'age': 35,
     'salary': 65000,
     'role': 'Senior Software Engineer',
     'description': "Leads software development projects and maintains company's digital platforms. Specializes in scalable backend systems."},
    {'name': 'Bob Thebuilder',
     'age': 25,
     'salary': 55000,
     'role': 'Junior IT Support Specialist',
     'description': 'Provides technical assistance and resolves IT-related issues for employees. Handles hardware and software troubleshooting.'}]},
  {'name': 'HR',
   'description': 'Responsible for talent acquisition, employee relations, and professional development.',
   'employees': [{'name': 'Carol Manager',
     'age': 45,
     'salary': 68000,
     'role': 'Expert HR Director',
     'description': 'Oversees all human resources functions, includi